In [5]:
import scipy.special
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from chaospy.quadrature import clenshaw_curtis

In [9]:
def wynn_epsilon(S):
    n = S.size
    width = n-1
    print(width)
    tableau = np.zeros((n + 1, width + 2))
    tableau[:,0] = 0
    tableau[1:,1] = S.copy() 
    for w in range(2,width + 2):
        for r in range(w,n+1):
            #print(r,w)
            tableau[r,w] = tableau[r-1,w-2] + 1/(tableau[r,w-1] - tableau[r-1,w-1])
    return tableau

# n = 30
# print(np.cumsum(np.arange(1,n,dtype="float")**-5))
# tableau = (wynn_epsilon(np.cumsum(np.arange(1,n,dtype="float")**-5)))
# plt.plot(np.abs(1.0369277551433699263-tableau[:,-1][tableau[:,-1]>0]),"o")
# plt.semilogy(np.abs(1.0369277551433699263-tableau[:,1]))
# print(tableau[:,3])
# print(tableau[:,5])


In [156]:
def wynn_epsilon_nonuniform(x, h):
    """
    Apply the Wynn-Epsilon acceleration method to a nonuniform sequence.
    
    Parameters:
    x (list or np.array): The input sequence to accelerate.
    h (list or np.array): The step sizes between consecutive terms in the sequence.
    
    Returns:
    np.array: The accelerated sequence after applying Wynn-Epsilon.
    """
    # Convert inputs to numpy arrays for easier manipulation
    x = np.array(x, dtype=float)
    h = np.array(h, dtype=float)
    N = len(x)
    
import numpy as np

def wynn_epsilon(sequence):
    """
    Apply the Wynn-Epsilon acceleration method to a sequence with uniform spacing.
    
    Parameters:
    sequence (list or np.array): The input sequence to accelerate.
    
    Returns:
    np.array: The accelerated sequence after applying Wynn-Epsilon.
    """
    # Convert sequence to numpy array for easier manipulation
    x = np.array(sequence, dtype=float)
    N = len(x)
    
    # Initialize the accelerated sequence, starting with the original sequence
    epsilon = np.copy(x)
    
    # Apply Wynn-Epsilon method recursively for the uniform case
    for p in range(1, N):  # Iterate over levels of recursion
        for q in range(N - p):  # Iterate over the indices in the sequence
            # Apply the recursion: ε^{p}_{q+1} = ε^{p+1}_{q-1} + (ε^{p+1}_q - ε^p_q)^{-1}
            epsilon[p, q] = epsilon[p + 1, q - 1] + (epsilon[p + 1, q] - epsilon[p, q]) ** (-1)
        
    # Return the accelerated sequence
    return epsilon

def wynn_epsilon_nonuniform(x, h):
    """
    Apply the Wynn-Epsilon acceleration method to a nonuniform sequence.
    
    Parameters:
    x (list or np.array): The input sequence to accelerate.
    h (list or np.array): The step sizes between consecutive terms in the sequence.
    
    Returns:
    np.array: The accelerated sequence after applying Wynn-Epsilon.
    """
    # Convert inputs to numpy arrays for easier manipulation
    x = np.array(x, dtype=float)
    h = np.array(h, dtype=float)
    N = len(x)
    
    # Initialize the table for the accelerated sequence
    epsilon = np.zeros((N, N), dtype=float)
    
    # First row is just the original sequence (p=0)
    epsilon[0, :N] = x
    
    # Apply Wynn-Epsilon method recursively for the nonuniform case
    for p in range(1, N-1):  # Iterate over levels of recursion
        for q in range(N - p):  # Iterate over the indices in the sequence
            n = q  # The starting index in the original sequence
            m = p + q  # The offset for the second term in the original sequence
            
            if m >= N:
                break  # Prevent out-of-bound errors
            
            # Generalized difference accounting for nonuniform spacing
            delta_n = (x[n+1] - x[n]) / h[n] if n+1 < N else 0
            delta_m = (x[m+1] - x[m]) / h[m] if m+1 < N else 0
            
            # Apply the generalized difference in the recursion step
            epsilon[p, q] = epsilon[p + 1, q + 1] + (delta_m - delta_n) ** (-1)
        
    # Return the final accelerated sequence (top-right corner of the table)
    return epsilon


# Example sequence with uniform spacing
# sequence = [1, 2, 3, 4, 5]
# accelerated_sequence = wynn_epsilon(sequence)
# print("Accelerated sequence (uniform):", accelerated_sequence)

# Example sequence with nonuniform step sizes
sequence_nonuniform = [1, 3, 6, 10, 15]  # Nonuniform increasing sequence
steps_nonuniform = [1, 2, 3, 5]  # Nonuniform step sizes
accelerated_sequence_nonuniform = wynn_epsilon_nonuniform(sequence_nonuniform, steps_nonuniform)
print("Accelerated sequence (nonuniform):", accelerated_sequence_nonuniform)

# Example




Accelerated sequence (nonuniform): [[ 1.          3.          6.         10.         15.        ]
 [-2.         -6.         -3.         -1.          0.        ]
 [-1.5        -2.         -0.75        0.          0.        ]
 [-1.         -0.66666667  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


In [158]:
def pi_approx(N):
    res = 0
    for i in range(N):
        res += (-3)**(-i)/(2*i + 1)
    return res * math.sqrt(12)

In [186]:
Ns = np.array([1,2,3,4,5])
pi_table = []
# hs = [100,2,1,10]
hs = np.ones(9)
for it in range(1, 10):

    # pi_table[it] = pi_approx(Ns[it])
    pi_table.append(pi_approx(it))
pi_table = [pi_approx(1), pi_approx(3), pi_approx(7), pi_approx(8)]
hs = [2, 4, 1]

In [188]:
wynn_epsilon_nonuniform(pi_table, hs) - math.pi

array([[ 3.22508962e-01,  1.45888180e-02,  8.16591090e-05,
        -2.39376480e-05],
       [ 3.51029432e+00,  2.80853068e+02,  9.46684620e+03,
        -3.14159265e+00],
       [ 3.35805580e+00,  2.72584346e+02, -3.14159265e+00,
        -3.14159265e+00],
       [-3.14159265e+00, -3.14159265e+00, -3.14159265e+00,
        -3.14159265e+00]])

In [190]:
pi_table[-1] - math.pi

-2.3937648009120238e-05